In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pytweening
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
plt.ioff()
plt.rc('font', size=16)
plt.rc('animation', html='html5')

In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def tween(x1, x2, fn, n_points=50):
    if np.isscalar(x1) and np.isscalar(x2):
        xx = np.linspace(x1, x2, n_points).reshape(-1, 1)
        scaler = MinMaxScaler().fit(xx)
        return scaler.inverse_transform(
            np.array([fn(x) for x in np.linspace(0, 1, n_points)]).reshape(-1, 1)
        )
    xx_linear = np.linspace(x1, x2, n_points)
    scaler = MinMaxScaler().fit(xx_linear)
    xx_minmax = scaler.transform(xx_linear)
    # because rounding, sometimes we end up w/ numbers like 1.0000000002
    xx_minmax = np.where(xx_minmax > 1, 1, xx_minmax)
    xx_minmax_t = pd.DataFrame(xx_minmax).apply(lambda col: [fn(x) for x in col]).values
    return scaler.inverse_transform(xx_minmax_t)

In [4]:
plt.ioff()
fps = 24

fig, ax = plt.subplots(figsize=(16, 9))

x1 = np.random.normal(5, 1, 25)
x2 = StandardScaler().fit_transform(x1.reshape(-1, 1)).ravel()
# xx = np.linspace(x1, x2, 24 * 3)
xx = tween(x1, x2, pytweening.easeInExpo, 24 * 3)

y1 = np.random.normal(5, 1, 25)
y2 = StandardScaler().fit_transform(y1.reshape(-1, 1)).ravel()
yy = tween(y1, y2, pytweening.easeInExpo, 24 * 3)

lines, = ax.plot(x1, y1, ls='', marker='.')
ax.set(ylim=(-4, 8), xlim=(-4, 8))
ax.hlines(0, -4, 8, ls=':', color='grey')
ax.vlines(0, -4, 8, ls=':', color='grey')

def animate(i):
    lines.set_data(xx[i], yy[i])
    
anim = FuncAnimation(fig, animate, interval=1000/fps, frames=24 * 3)
anim